## 台北房價預測

請撰寫程式讀取台北市房價資料集 `Taipei_house.csv`，建立迴歸分析模型預測房價，並分析其效能。

## 欄位說明
| 編號 | 欄位名稱 | 說明 |
|:---:|:---:|:---:|
| 1 | 行政區* | 包含 4 個區域 |
| 2 | 土地面積 | 平方公尺 |
| 3 | 建物總面積 | 平方公尺 |
| 4 | 屋齡 | 年 |
| 5 | 樓層 |  |
| 6 | 總樓層 |  |
| 7 | 用途* | 0: 住宅用、1: 商業用 |
| 8 | 房數 |  |
| 9 | 廳數 |  |
| 10 | 衛數 |  |
| 11 | 電梯* | 0: 無、1: 有 |
| 12 | 車位類別* | 共 8 種 |
| 13 | 交易日期 | 年月日 |
| 14 | 經度 |  |
| 15 | 緯度 |  |
| 16 | 總價 | 萬元 |

備註：欄位有標示星號（*）者為類別變數，其餘為數值變數。

## 分析程序
1. 進行「行政區」欄位的獨熱編碼（One-Hot encoding），並修改「車位類別」欄位：將欄位值="無"修改為 0，其餘為 1。
2. 請將資料集分為訓練集與測試集，其中測試集佔 20%。
3. 建立四個迴歸模型，包括多元迴歸、脊迴歸、多項式迴歸、多項式迴歸 + L1 正則化。
4. 針對訓練集的 15 個欄位（編號 2 ~ 12 欄位 + 行政區的獨熱編碼 4 個欄位），擬合上述四個迴歸模型，並分別對訓練集與測試集，計算均方根誤差（Root Mean Square Error, RMSE）。以及調整後的 R 平方（Adjusted R-squared）。
    1. 輸出四個迴歸模型對訓練集的最大 Adjusted R-squared。
    2. 輸出四個迴歸模型對測試集的最小 RMSE（計算至整數）。
    3. 輸出兩個迴歸模型（多元迴歸與脊迴歸）對測試集的最大 Adjusted R-squared。
5. 利用所有資料重新擬合模型，輸入一個屋況進行房價預測。
    1. 浮點數均四捨五入取至小數點後第四位。
    2. 輸入一屋況如下，使用四個模型中對測試集有最大 Adjusted R-squared 的模型，再重新擬合所有資料後，預測其房價為何（計算至整數）
        | 欄位名稱 | 說明 |
        |:---:|:---:|
        | 行政區 | 松山區 |
        | 土地面積 | 36 |
        | 建物總面積 | 99 |
        | 屋齡 | 32 |
        | 樓層 | 4 |
        | 總樓層 | 4 |
        | 用途 | 住宅用 |
        | 房數 | 3 |
        | 廳數 | 2 |
        | 衛數 | 1 |
        | 電梯 | 無 |
        | 車位類別 | 無 |

In [1]:
# #############################################################################
# 本題參數設定，請勿更改
seed = 0   # 亂數種子數  
# #############################################################################
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd 

# 初始化放置評估結果的 DataFrame
dct = {'模型': [], '細節':[], 'RMSE (test)':[], 
       'R2 (train)':[], 'adj. R2 (train)':[], 
       'R2 (test)':[], 'adj. R2 (test)':[]}
df_eval = pd.DataFrame(dct)

# 讀取台北市房價資料集
df = pd.read_csv('Taipei_house.csv')

# 對"行政區"進行 one-hot encoding
df = pd.get_dummies(df, columns=['行政區'])

# 處理"車位類別" (將欄位值="無"轉換為0, 其他轉換為1)
df['車位類別'] = [0 if x=='無' else 1 for x in df['車位類別']]

# 檢視 dataframe 前3筆資料
df.head(3)

,土地面積,建物總面積,屋齡,樓層,總樓層,用途,房數,廳數,衛數,電梯,車位類別,交易日期,經度,緯度,總價,行政區_信義區,行政區_大安區,行政區_文山區,行政區_松山區
0,33.81,109.42,38.996009,3,5,0,3,2,2,0,0,2019/10/6,121.552517,25.004507,1000,False,False,True,False
1,32.19,163.53,23.781460,3,11,0,3,2,2,1,1,2019/4/28,121.559133,24.983199,2100,False,False,True,False
2,60.25,204.79,0.744711,1,10,0,3,2,3,1,0,2019/10/23,121.576052,24.988665,6720,False,False,True,False


In [2]:
from sklearn.model_selection import train_test_split


# 移除掉不需要的欄位
features = df.drop(['交易日期', '經度', '緯度', '總價'], axis=1).columns
target = '總價'

# 切分訓練集(80%)、測試集(20%)
X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    test_size=0.2, 
    random_state=seed
)

X_train.shape

(9943, 15)

In [3]:
# 計算 Adjusted R-squared
def adj_R2(r2, n, k):
    return r2-(k-1)/(n-k)*(1-r2)

In [4]:
from sklearn.metrics import mean_squared_error

# 定義測量指標
def measurement(model, X_train, X_test, y_train, y_test):
    # 預測測試資料集
    y_pred = model.predict(X_test)
    
    # 計算 RMSE
    rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 0)
    
    # 計算訓練資料的 R2
    r2_train = round(model.score(X_train, y_train), 4)
    
    # 計算訓練資料的 Adjusted R2
    adj_r2_train = round(
        adj_R2(
            model.score(X_train, y_train), 
            X_train.shape[0], 
            X_train.shape[1]
        ), 
        4 # 四捨五入到小數點後四位
    )
    
    # 計算測試資料的 R2
    r2_test = round(model.score(X_test, y_test), 4)
    
    # 計算測試資料的 Adjusted R2
    adj_r2_test = round(
        adj_R2(
            model.score(X_test, y_test), 
            X_test.shape[0], 
            X_test.shape[1]
        ), 
        4 # 四捨五入到小數點後四位
    )
    
    return [rmse, r2_train, adj_r2_train, r2_test, adj_r2_test]

In [5]:
# 整合所有模型與對應的資訊
list_model, list_info = [], []

In [6]:
# 多元迴歸(參數皆為預設值)
# #########################################################################
# '行政區_信義區', '行政區_大安區', '行政區_文山區','行政區_松山區' 四個特徵是經過
# one-hot encoding 後產生，若欄位名稱不同可自行修改之。
# #########################################################################
from sklearn import linear_model
list_model.append(linear_model.LinearRegression())
list_info.append(['多元迴歸', '15 features'])

In [7]:
# 脊迴歸(Ridge regression)，除以下參數設定外，其餘為預設值
# #########################################################################
# alpha=10
# #########################################################################
list_model.append(linear_model.Ridge(alpha=10))
list_info.append(['Ridge', '15 features'])

In [8]:
# 多項式迴歸，除以下參數設定外，其餘為預設值
# #########################################################################
# degree=2
# #########################################################################
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.fit_transform(X_test)

list_model.append(linear_model.LinearRegression())
list_info.append(['多項式迴歸', 'deg=2'])

In [9]:
# 多項式迴歸 + L1正規化，除以下參數設定外，其餘為預設值
# #########################################################################
# alpha=10
# #########################################################################
list_model.append(linear_model.Lasso(alpha=10))
list_info.append(['多項式迴歸+L1正規化', 'deg=2'])

In [10]:
# 將每一個模型進行訓練後，對資料集進行計算並儲存評估結果
for i in range(len(list_model)):
    # 如果是多項式迴歸，則使用多項式特徵
    if '多項式' in list_info[i][0]:
        X_train, X_test = X_train_poly, X_test_poly
    
    # 訓練模型
    model = list_model[i].fit(X_train, y_train)

    # 計算並儲存評估結果
    row = list_info[i] + measurement(model, X_train, X_test, y_train, y_test)
    
    # 將結果加入 DataFrame
    df_eval.loc[i] = row

print('對訓練集的最大 Adjusted R-squared: {}'.format(max(df_eval['adj. R2 (train)'])))
print('對測試集的最小 RMSE: {}'.format(min(df_eval['RMSE (test)'])))
print('兩個模型對測試集的最大 Adjusted R-squared: {}'.format(max(df_eval.loc[:1, 'adj. R2 (test)'])))

對訓練集的最大 Adjusted R-squared: 0.9252
對測試集的最小 RMSE: 801.0
兩個模型對測試集的最大 Adjusted R-squared: 0.8046


In [11]:
# 儲存評估結果
df_eval

,模型,細節,RMSE (test),R2 (train),adj. R2 (train),R2 (test),adj. R2 (test)
0,多元迴歸,15 features,1069.0,0.8106,0.8103,0.8056,0.8045
1,Ridge,15 features,1069.0,0.8106,0.8103,0.8057,0.8046
2,多項式迴歸,deg=2,807.0,0.9262,0.9252,0.8891,0.8828
3,多項式迴歸+L1正規化,deg=2,801.0,0.9227,0.9216,0.8909,0.8846


In [12]:
''' 預測 '''
# 利用所有資料重新擬合模型，並進行預測
X = df[features]
y = df[target]
X_poly = poly_features.fit_transform(X)

#features= ['土地面積', '建物總面積', '屋齡', '樓層', '總樓層', '用途', 
#           '房數', '廳數', '衛數', '電梯', '車位類別', 
#           '行政區_信義區', '行政區_大安區', '行政區_文山區','行政區_松山區']
new = np.array([36, 99, 32, 4, 4, 0, 3, 2, 1, 0, 0, 0, 0, 0, 1]).reshape(1, -1)

# 將新資料轉換為多項式特徵
df_new = pd.DataFrame(new, columns=features)
df_poly_features = poly_features.fit_transform(df_new)

# 找出測試集中 Adjusted R-squared 最大的模型
lst = df_eval['adj. R2 (test)'].tolist()
'''
[0.8045, 0.8046, 0.8828, 0.8846]
'''

# 找出成效最佳模型的索引
idx = lst.index(max(lst))

# 不是多項式迴歸，就使用原始特徵
if idx <=1:
    model = list_model[idx].fit(X, y)
    print('房價預測結果：%d' % model.predict(df_new))
else: # 如果是多項式迴歸，則使用多項式特徵
    model = list_model[idx].fit(X_poly, y)
    print('房價預測結果：%d' % model.predict(df_poly_features))

房價預測結果：1546
